In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from collections import Counter

In [7]:
def get_df_PER(df):
    df_BPER = df[df['Tag'] == 'B-PER']
    df_SPER = df[df['Tag'] == 'S-PER']
    
    df_PER_1 = pd.concat([df_BPER, df_IPER])
    df_PER_2 = pd.concat([df_EPER, df_SPER])
    
    df_PER = pd.concat([df_PER_1, df_PER_2])
    df_PER = df_PER.sort_index(ascending=True)
    return df_PER

In [8]:
def get_counter(list_tag, N):
    list_elem_cleared = [elem for elem in list_tag if elem is not None]
    return Counter(list_elem_cleared).most_common(N)

In [9]:
def get_PER_surrounding(list_mention_token, list_head_token):
    list_surronding_PER = []
    for i in range(len(list_mention_token)):
        if list_mention_token[i] == 'B-PER' or 'S-PER':
            list_surronding_PER.append(list_head_token[i])
    return list_surronding_PER

In [22]:
def get_N_surronding_tokens(index, df_courant, N):
    list_token = []
    list_mention_token = []
    list_head_token = []
    print(len(index))
    
    for index_mention in index:
        if int(index_mention) > N:
            for i in range(index_mention-N, index_mention+N+1):
                list_token.append(df_courant.loc[[i]]['Token'].values[0])
                list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])
        else:
            for i in range(index_mention, index_mention+N+1):
                list_token.append(df_courant.loc[[i]]['Token'].values[0])
                list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])
               
    return list_token, list_mention_token, list_head_token

In [2]:
def get_dict_surronding_PER(df_courant, N_PER, N_token):
    dict_surronding_PER = {}
    df_PER = get_df_PER(df_courant)
    dict_mention = get_counter(list(df_PER['Head_tag']), N_PER)
    df_mention_roman = pd.DataFrame(dict_mention, columns=['mention', 'count'])
    
    for mention in list(df_mention_roman['mention'])[1:]:# exit nan value
        if mention in list(df_courant['Head_tag']):
            # get_index_mention
            df_mentionned = df_courant[df_courant['Head_tag'] == mention]
            index = df_mentionned.index
            list_surronding_token, list_surronding_mention_token, list_surronding_head_token = get_N_surronding_tokens(index, df_courant, N_token)
            list_surronding_PER = get_PER_surrounding(list_surronding_mention_token, list_surronding_head_token)
        dict_surronding_PER[mention] = list_surronding_PER
    return dict_surronding_PER         

In [3]:
def mouli_preds(path_name, N_PER, N_token):
    str_base = '_'
    for doc in tqdm(glob(path_name)):
        print(doc)
        df_courant = pd.read_csv(doc)
        dict_surronding_PER = get_dict_surronding_PER(df_courant, N_PER, N_token)
        print(dict_surronding_PER)
        df_res = pd.DataFrame(dict_surronding_PER)        
        temp = str(path_name).split("_")[2:4]
        df_res.to_csv(r'res/'+str_base.join(temp)+'.csv')
        
    return df_res

In [24]:
path_name = "data/*.csv"
path_test = "data_test/*.csv"
N_PER = 20
N_token = 100

In [23]:
df_res = mouli_preds(path_test, N_PER, N_token)

  0%|          | 0/2 [00:00<?, ?it/s]

data_test\Adam-Paul_La-glebe.csv
198
111
110
88


KeyError: "None of [Int64Index([10286], dtype='int64')] are in the [index]"